__Subida de datos a [doccano](https://doccano.github.io/doccano/)__

__Paso 1:__
* Instalar la libreria: ```!pip install doccano```
* A continuación, iniciar el servidor:

```
# Initialize database. First time only.
doccano init
# Create a super user. First time only.
doccano createuser --username admin --password pass
# Start a web server.
doccano webserver --port 8000
```

* En otro terminal, ejecutar el siguiente comando:

```
doccano task
```

De cara a subir el dataset, seguir [los siguientes pasos](https://doccano.github.io/doccano/tutorial/).

In [76]:
# -- Convertimos la salida del modelo de OpenAI en el formato de doccano
import pandas as pd
import re
import os
output_df = pd.read_json("/Users/alberto/Personal Projects/hackathon_podcast/data/NER/json_entity/json_files_alberto/output_17.json")
output_df['fe_entities'] = output_df.apply(lambda x: [(x_aux['class'], x_aux['text'].lower(), re.search(x_aux['text'].lower(), x['text'].lower())) for x_aux in x['entities']\
                                                    if x_aux['text'] != "" and x_aux['text'] != None and x_aux['text'] not in ['?', '!', '¿', '?']], axis=1)
output_df['fe_entities_start_end'] = output_df['fe_entities'].apply(lambda x: [[x_aux[-1].start(), x_aux[-1].end(), x_aux[0].upper()] for x_aux in x if x_aux[-1] != None])

__Doccano format__:

```
{"text": "EU rejects German call to boycott British lamb.", "label": [[0, 2, "ORG"]]}
{"text": "Peter Blackburn", "label": [[0, 15, "PERSON"]]}
{"text": "President Obama", "label": [[10, 15, "PERSON"]]}
```

In [77]:
output_df['fe_final_dict'] = output_df.apply(lambda x: {"text": x["text"], "label": x["fe_entities_start_end"]}, axis=1)
output_df

,text,entities,fe_entities,fe_entities_start_end,fe_final_dict
0,¿Tú eres un rockero metalero auténtico?\n,"[{'class': 'people', 'text': 'Tú'}]","[(people, tú, <re.Match object; span=(1, 3), m...","[[1, 3, PEOPLE]]",{'text': '¿Tú eres un rockero metalero auténti...
1,"Me gusta un poco el metal, pero lo que más me ...","[{'class': 'topics', 'text': 'metal'}, {'class...","[(topics, metal, <re.Match object; span=(20, 2...","[[20, 25, TOPICS], [94, 102, TOPICS]]","{'text': 'Me gusta un poco el metal, pero lo q..."
2,"Sí, sí, sí, me encanta el suicidal, me encanta.\n","[{'class': 'songs', 'text': 'suicidal'}]","[(songs, suicidal, <re.Match object; span=(26,...","[[26, 34, SONGS]]","{'text': 'Sí, sí, sí, me encanta el suicidal, ..."
3,"Metal crossover, se pasaron... Me gusta mucho ...","[{'class': 'topics', 'text': 'Metal crossover'...","[(topics, metal crossover, <re.Match object; s...","[[0, 15, TOPICS], [49, 57, TOPICS], [61, 87, D...","{'text': 'Metal crossover, se pasaron... Me gu..."
4,"Me encanta Dead Kennins, me encanta... Ah, no,...","[{'class': 'people', 'text': 'Dead Kennins'}, ...","[(people, dead kennins, <re.Match object; span...","[[11, 23, PEOPLE], [11, 23, SONGS]]","{'text': 'Me encanta Dead Kennins, me encanta...."
...,...,...,...,...,...
59,"¿Sabiste el rollo californiano de punk, skate ...","[{'class': 'topics', 'text': 'Rollo California...","[(topics, rollo californiano, <re.Match object...","[[12, 30, TOPICS], [34, 38, TOPICS], [40, 50, ...",{'text': '¿Sabiste el rollo californiano de pu...
60,"Me da muy buen rollo, me dan alegría, ¿sabes?\n","[{'class': 'topics', 'text': 'buen rollo'}, {'...","[(topics, buen rollo, <re.Match object; span=(...","[[10, 20, TOPICS], [29, 36, TOPICS]]","{'text': 'Me da muy buen rollo, me dan alegría..."
61,"Sí, porque es música agresiva y todo, pero al ...","[{'class': 'places', 'text': 'California'}]","[(places, california, <re.Match object; span=(...","[[55, 65, PLACES]]","{'text': 'Sí, porque es música agresiva y todo..."
62,Como estar en California es como que tiene un ...,"[{'class': 'places', 'text': 'California'}]","[(places, california, <re.Match object; span=(...","[[14, 24, PLACES]]",{'text': 'Como estar en California es como que...


__Por último, exportamos el diccionario a .jsonl__

In [72]:
# Abre un archivo de salida en modo de escritura
import json
with open('/Users/alberto/Desktop/output.jsonl', 'w') as f:
    # Itera a través de las filas del DataFrame
    for _, row in output_df.iterrows():
        # Convierte el diccionario en una cadena JSON
        json_str = json.dumps(row['fe_final_dict'])
        # Escribe la cadena JSON en el archivo de salida, seguida de un salto de línea
        f.write(json_str + '\n')